In [1]:
import imblearn
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import os
from math import ceil 

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE


In [2]:
customer_df=pd.read_csv(r"C:\Users\Quaresma\Documents\IRONHACK\Labs\12 lab-cleaning-numerical-data\files_for_lab\we_fn_use_c_marketing_customer_value_analysis.csv")


In [3]:
col=[customer_df.columns[i].lower().replace(" ","_") for i in range(len(customer_df.columns))]
customer_df.columns=col
customer_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,LA72316,California,23405.987980,No,Basic,Bachelor,2/10/11,Employed,M,71941,...,89,0,2,Personal Auto,Personal L1,Offer2,Web,198.234764,Four-Door Car,Medsize
9130,PK87824,California,3096.511217,Yes,Extended,College,2/12/11,Employed,F,21604,...,28,0,1,Corporate Auto,Corporate L3,Offer1,Branch,379.200000,Four-Door Car,Medsize
9131,TD14365,California,8163.890428,No,Extended,Bachelor,2/6/11,Unemployed,M,0,...,37,3,2,Corporate Auto,Corporate L2,Offer1,Branch,790.784983,Four-Door Car,Medsize
9132,UP19263,California,7524.442436,No,Extended,College,2/3/11,Employed,M,21941,...,3,0,3,Personal Auto,Personal L2,Offer3,Branch,691.200000,Four-Door Car,Large


In [4]:
customer_df.select_dtypes(include=[np.number]).dtypes

customer_lifetime_value          float64
income                             int64
monthly_premium_auto               int64
months_since_last_claim            int64
months_since_policy_inception      int64
number_of_open_complaints          int64
number_of_policies                 int64
total_claim_amount               float64
dtype: object

### 1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [5]:
def outliers(df):
    global df_all_no_outliers
    numerical_df= df.select_dtypes(include=np.number)
    categorical_df= df.select_dtypes(include=np.object)
    q25 = numerical_df.quantile(.25)
    q75 = numerical_df.quantile(.75)
    iqr = (q75 - q25)

    upper_lim = q75 + 3 * iqr
    lower_lim = q25 - 3 * iqr

    filters2 = []
    for metric in numerical_df.columns:
        llim = lower_lim[metric]
        ulim = upper_lim[metric]
        filters2.append(numerical_df[metric].between(llim, ulim, inclusive=True))

    filters2 = pd.Series(np.all(filters2, 0))
    df_no_outliers = numerical_df[filters2]
    
    print('Percentage of data kept after removing outliers:', np.round(df_no_outliers.shape[0] / numerical_df.shape[0], 4))
    df_all_no_outliers= pd.merge(df_no_outliers, categorical_df,left_index=True, right_index=True)
    return df_all_no_outliers
outliers(customer_df)



Percentage of data kept after removing outliers: 0.7487


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,customer,state,...,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,2763.519279,56274,69,32,5,0,1,384.811147,BU79786,Washington,...,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,6979.535903,0,94,13,42,0,8,1131.464935,QZ44356,Arizona,...,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,12887.431650,48767,108,18,38,0,2,566.472247,AI49188,Nevada,...,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,7645.861827,0,106,18,65,0,7,529.881344,WW63253,California,...,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,2813.692575,43836,73,12,44,0,1,138.130879,HB64268,Washington,...,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,4100.398533,47761,104,16,58,0,1,541.282007,YM19146,California,...,Employed,F,Suburban,Single,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Large
9129,23405.987980,71941,73,18,89,0,2,198.234764,LA72316,California,...,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,3096.511217,21604,79,14,28,0,1,379.200000,PK87824,California,...,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9132,7524.442436,21941,96,34,3,0,3,691.200000,UP19263,California,...,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


### 2. Create a copy of the dataframe for the data wrangling.

In [6]:
data_wrangling= df_all_no_outliers
data_wrangling

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,customer,state,...,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,2763.519279,56274,69,32,5,0,1,384.811147,BU79786,Washington,...,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,6979.535903,0,94,13,42,0,8,1131.464935,QZ44356,Arizona,...,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,12887.431650,48767,108,18,38,0,2,566.472247,AI49188,Nevada,...,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,7645.861827,0,106,18,65,0,7,529.881344,WW63253,California,...,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,2813.692575,43836,73,12,44,0,1,138.130879,HB64268,Washington,...,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,4100.398533,47761,104,16,58,0,1,541.282007,YM19146,California,...,Employed,F,Suburban,Single,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Large
9129,23405.987980,71941,73,18,89,0,2,198.234764,LA72316,California,...,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,3096.511217,21604,79,14,28,0,1,379.200000,PK87824,California,...,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9132,7524.442436,21941,96,34,3,0,3,691.200000,UP19263,California,...,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


### 3. Normalize the continuous variables. You can use any one method you want.

In [7]:
dt_w_num= data_wrangling.select_dtypes(include=np.number)


continuous_variables=['customer_lifetime_value','income','monthly_premium_auto','total_claim_amount']

scaler = MinMaxScaler()
dt_w_num[continuous_variables] = scaler.fit_transform(dt_w_num[continuous_variables])
dt_w_num

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0.035233,0.562847,0.046784,32,5,0,1,0.280166
1,0.228462,0.000000,0.192982,13,42,0,8,0.823915
2,0.499234,0.487763,0.274854,18,38,0,2,0.412460
3,0.259001,0.000000,0.263158,18,65,0,7,0.385813
4,0.037533,0.438443,0.070175,12,44,0,1,0.100521
...,...,...,...,...,...,...,...,...
9128,0.096505,0.477701,0.251462,16,58,0,1,0.394115
9129,0.981322,0.719547,0.070175,18,89,0,2,0.144292
9130,0.050495,0.216081,0.105263,14,28,0,1,0.276079
9132,0.253437,0.219452,0.204678,34,3,0,3,0.503293


### 4. Encode the categorical variables

In [8]:
data_wrangling.select_dtypes(include= object).dtypes
dt_w_cat=data_wrangling.select_dtypes(include= object)
dt_w_cat

,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,YM19146,California,No,Premium,College,1/6/11,Employed,F,Suburban,Single,Personal Auto,Personal L2,Offer1,Branch,Four-Door Car,Large
9129,LA72316,California,No,Basic,Bachelor,2/10/11,Employed,M,Urban,Married,Personal Auto,Personal L1,Offer2,Web,Four-Door Car,Medsize
9130,PK87824,California,Yes,Extended,College,2/12/11,Employed,F,Suburban,Divorced,Corporate Auto,Corporate L3,Offer1,Branch,Four-Door Car,Medsize
9132,UP19263,California,No,Extended,College,2/3/11,Employed,M,Suburban,Married,Personal Auto,Personal L2,Offer3,Branch,Four-Door Car,Large


In [9]:

ordi= dt_w_cat[['coverage', 'employmentstatus', 'location_code', 'vehicle_size']]


for column in ordi.columns:
    unique = set(dt_w_cat[column])
    print(column,':',unique)

coverage : {'Basic', 'Premium', 'Extended'}
employmentstatus : {'Disabled', 'Unemployed', 'Retired', 'Medical Leave', 'Employed'}
location_code : {'Rural', 'Suburban', 'Urban'}
vehicle_size : {'Medsize', 'Small', 'Large'}


In [10]:
dt_w_cat["coverage"] = dt_w_cat["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2}).astype(int)
dt_w_cat["vehicle_size"] = dt_w_cat["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2}).astype(int)

dummies=[x for x in dt_w_cat.columns if x not in ['customer','effective_to_date']]


dt_w_cat = pd.get_dummies(dt_w_cat, columns= dummies)
dt_w_cat





,customer,effective_to_date,state_Arizona,state_California,state_Nevada,state_Oregon,state_Washington,response_No,response_Yes,coverage_0,...,sales_channel_Web,vehicle_class_Four-Door Car,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_0,vehicle_size_1,vehicle_size_2
0,BU79786,2/24/11,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,1,0,1,0
1,QZ44356,1/31/11,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,AI49188,2/19/11,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,WW63253,1/20/11,0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0
4,HB64268,2/3/11,0,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,YM19146,1/6/11,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
9129,LA72316,2/10/11,0,1,0,0,0,1,0,1,...,1,1,0,0,0,0,0,0,1,0
9130,PK87824,2/12/11,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
9132,UP19263,2/3/11,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1


### 5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [11]:
dt_w_cat['effective_to_date'] = pd.to_datetime(dt_w_cat['effective_to_date'])
dt_w_cat['month']= dt_w_cat['effective_to_date'].dt.month
dt_w_cat['day']= dt_w_cat['effective_to_date'].dt.day
dt_w_cat['year']= dt_w_cat['effective_to_date'].dt.year

dt_w_cat = dt_w_cat.drop(['effective_to_date'], axis =1)

### 6.Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [12]:
df_final= pd.merge(dt_w_num, dt_w_cat,left_index=True, right_index=True)
df_final

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount,customer,state_Arizona,...,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_0,vehicle_size_1,vehicle_size_2,month,day,year
0,0.035233,0.562847,0.046784,32,5,0,1,0.280166,BU79786,0,...,0,0,0,1,0,1,0,2,24,2011
1,0.228462,0.000000,0.192982,13,42,0,8,0.823915,QZ44356,1,...,0,0,0,0,0,1,0,1,31,2011
2,0.499234,0.487763,0.274854,18,38,0,2,0.412460,AI49188,0,...,0,0,0,1,0,1,0,2,19,2011
3,0.259001,0.000000,0.263158,18,65,0,7,0.385813,WW63253,0,...,0,1,0,0,0,1,0,1,20,2011
4,0.037533,0.438443,0.070175,12,44,0,1,0.100521,HB64268,0,...,0,0,0,0,0,1,0,2,3,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,0.096505,0.477701,0.251462,16,58,0,1,0.394115,YM19146,0,...,0,0,0,0,0,0,1,1,6,2011
9129,0.981322,0.719547,0.070175,18,89,0,2,0.144292,LA72316,0,...,0,0,0,0,0,1,0,2,10,2011
9130,0.050495,0.216081,0.105263,14,28,0,1,0.276079,PK87824,0,...,0,0,0,0,0,1,0,2,12,2011
9132,0.253437,0.219452,0.204678,34,3,0,3,0.503293,UP19263,0,...,0,0,0,0,0,0,1,2,3,2011


In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6839 entries, 0 to 9133
Data columns (total 69 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   customer_lifetime_value         6839 non-null   float64
 1   income                          6839 non-null   float64
 2   monthly_premium_auto            6839 non-null   float64
 3   months_since_last_claim         6839 non-null   int64  
 4   months_since_policy_inception   6839 non-null   int64  
 5   number_of_open_complaints       6839 non-null   int64  
 6   number_of_policies              6839 non-null   int64  
 7   total_claim_amount              6839 non-null   float64
 8   customer                        6839 non-null   object 
 9   state_Arizona                   6839 non-null   uint8  
 10  state_California                6839 non-null   uint8  
 11  state_Nevada                    6839 non-null   uint8  
 12  state_Oregon                    68

In [14]:
df_final.select_dtypes(include= object)
df_final = df_final.drop(['customer'], axis =1)
df_final.select_dtypes(include= object)

""
0
1
2
3
4
...
9128
9129
9130
9132


In [16]:
df_final.to_csv('final_customer.csv', index=True)